# Constrained dynamics
This tutorial is a follow-up of the notebook 2. We now write a complete optimal control problem, where both the states and the controls are decided. In a first time, we work with a simple manipulator robot in free space. The dynamics can then be evaluated directly with the ABA algorithm. Then, we move to a humanoid robot with floating actuation and constrained dynamics, and use the corresponding algorithm of Pinocchio for that.


In [1]:
import magic_donotload

NB: as for all the tutorials, a magic command %do_not_load is introduced to hide the solutions to some questions. Change it for %load if you want to see (and execute) the solution.


## Set up
We will use the following tools:
- the ur10 model (loaded by example-robot-data)
- pinocchio.casadi for writing the problem and computing its derivatives
- the IpOpt solver wrapped in casadi
- the meshcat viewer
- the pinocchio dynamics algorithms, namely ABA and constraintDynamics

In [2]:
# %load tp3/generated/free_import
import time
import unittest

import casadi
import example_robot_data as robex
import numpy as np
import pinocchio as pin
from pinocchio import casadi as cpin

from utils.meshcat_viewer_wrapper import MeshcatVisualizer

The notebook follows the same structure as notebook 2. Let's start with the manipulator robot.

## Free dynamics
We first write a manipulation problem, without contact. We search for the robot state trajectories $X=[x_0 ... x_T]$, with $x=(q,v)$ and control trajectories $U = [u_0 ... u_{T-1} ]$. 
We consider yet the robot to be fully actuatated, i.e. $u=\tau$ are the joint torques. 

The state evolution must be decided from the acceleration resulting from the actuation. This can be evaluated by the ABA algorithm.
$$ \forall t=0..T-1, a_t = ABA(q_t,v_t,\tau_t) $$
which is most commonly encountered under the matrix form $a_t = M(q_t)^{-1} ( \tau_t - b(q_t,v_t) )$, with $M$ the generalized mass matrix and $b$ the sum of nonlinear effects (coriolis, centrifugal, gravity).

To simplify the writing, we will consider $A = [a_0 ... a_{T-1} ]$ to be part of the decision variables. Strictly speaking, we call this kind of variables a **slack**, as it is introduced for convenience (often for the convenience of the solver, here for the convenience of the syntax). The optimal control problem then writes:

Decide:
- $X = [ x_0 ... x_T ]$ the state trajectory (with $x = (robot.q0 + dq, v)$)
- $A = [a_0 ... a_{T-1} ]$ the accelerations
- $U = [u_0 ... u_{T-1} ]$ the controls (joint torques)

Minimizing: $\sum_{t=0}^{T-1} v_t^2  + a_t^2 + e(q_T)^2$

Subject to:
- $x_0 = (robot.q0, 0)$
- $\forall t=0..T-1$
$$ a_t = ABA(q_t,v_t,\tau_t) $$
$$ v_{t+1} = v_t + a_t \Delta t$$
$$ q_{t+1} = q_t \oplus v_{t+1} \Delta t$$


### Set up the model
We simply copy the code of the previous notebook to load and display the robot.
The robot is loaded with example-robot-data, but feel free to load your own.

In [3]:
# %load tp3/generated/free_ur10
robot = robex.load("ur10")
# Open the viewer
viz = MeshcatVisualizer(robot)
viz.display(robot.q0)

# The pinocchio model is what we are really interested by.
model = robot.model
data = model.createData()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


The important frames are the effector and the target.

In [4]:
# %load tp3/generated/free_frames
Mtarget = pin.SE3(pin.utils.rotate("y", 3),
                  np.array([-0.5, 0.1, 0.2]))  # x,y,z
endEffectorFrameName = "tool0"
q0 = np.array([0, -3.14 / 2, 0, 0, 0, 0])

In [5]:
# %load tp3/generated/free_framesId
endEffector_ID = model.getFrameId(endEffectorFrameName)

The hyperparameters are defined as follow:

In [6]:
# %load tp3/generated/free_hyper
T = 50
DT = 0.002
w_vel = 0.1
w_conf = 5

Activate the visualization.

In [7]:
# %load tp3/generated/free_viz
# --- Add box to represent target
# Add a vizualization for the target
boxID = "world/box"
viz.addBox(boxID, [0.05, 0.1, 0.2], [1.0, 0.2, 0.2, 0.5])
# Add a vizualisation for the tip of the arm.
tipID = "world/blue"
viz.addBox(tipID, [0.08] * 3, [0.2, 0.2, 1.0, 0.5])


def displayScene(q, dt=1e-1):
    """
    Given the robot configuration, display:
    - the robot
    - a box representing endEffector_ID
    - a box representing Mtarget
    """
    pin.framesForwardKinematics(model, data, q)
    M = data.oMf[endEffector_ID]
    viz.applyConfiguration(boxID, Mtarget)
    viz.applyConfiguration(tipID, M)
    viz.display(q)
    time.sleep(dt)


def displayTraj(qs, dt=1e-2):
    for q in qs[1:]:
        displayScene(q, dt=dt)


displayScene(robot.q0)

In [8]:
viz.viewer.jupyter_cell()

Build Pinocchio Casadi model and data. This time, we need to compute the graph for the ABA algorithm. ABA takes the robot joint positions, velocities and torques, and returns the robot joint accelerations. An extended version also takes disturbance forces (expressed at every joint), but we will not use it here.

In [9]:
# %load tp3/generated/free_helpers
# --- Casadi helpers
cmodel = cpin.Model(model)
cdata = cmodel.createData()

nq = model.nq
nv = model.nv
nx = nq + nv
ndx = 2 * nv
cx = casadi.SX.sym("x", nx, 1)
cdx = casadi.SX.sym("dx", nv * 2, 1)
cq = cx[:nq]
cv = cx[nq:]
caq = casadi.SX.sym("a", nv, 1)
ctauq = casadi.SX.sym("tau", nv, 1)

# Compute kinematics casadi graphs
cpin.aba(cmodel, cdata, cq, cv, ctauq)
cpin.forwardKinematics(cmodel, cdata, cq, cv, caq)
cpin.updateFramePlacements(cmodel, cdata)


We cast ABA in a function to skip the SX/MX casadi syntax.

In [10]:
# %load tp3/generated/free_aba
# Sym graph for the aba operation
caba = casadi.Function("aba", [cx, ctauq], [cdata.ddq])

Similarly, we introduce a helper for the integration of the acceleration, as in the previous notebook.

In [11]:
# %load tp3/generated/free_cnext
# Sym graph for the integration operation x' = [ q+vDT+aDT**2, v+aDT ]
cnext = casadi.Function(
    "next",
    [cx, caq],
    [
        casadi.vertcat(
            cpin.integrate(cmodel, cx[:nq], cx[nq:] * DT + caq * DT**2),
            cx[nq:] + caq * DT,
        )
    ],
)

Finally, we need a helper for the operational target, as before.

In [12]:
# %load tp3/generated/free_error
# Sym graph for the operational error
error_tool = casadi.Function(
    "etool3", [cx], [
        cdata.oMf[endEffector_ID].translation - Mtarget.translation]
)

### Write and solve
We are now ready to formulate the problem. Follow the step for that.

1. Define the variables $X,A,U$.

In [13]:
# %load tp3/generated/free_ocp1
opti = casadi.Opti()
var_xs = [opti.variable(nx) for t in range(T + 1)]
var_as = [opti.variable(nv) for t in range(T)]
var_us = [opti.variable(nv) for t in range(T)]

2. Compute the cost

In [14]:
# %load tp3/generated/free_ocp2
totalcost = 0
# Define the running cost
for t in range(T):
    totalcost += 1e-3 * DT * casadi.sumsqr(var_xs[t][nq:])
    totalcost += 1e-4 * DT * casadi.sumsqr(var_as[t])
totalcost += 1e4 * casadi.sumsqr(error_tool(var_xs[T]))

3. Enforce the initial constraint. You can also add a terminal velocity constraint if you like.

In [15]:
# %load tp3/generated/free_ocp3
opti.subject_to(var_xs[0][:nq] == robot.q0)
opti.subject_to(var_xs[0][nq:] == 0)  # zero initial velocity
opti.subject_to(var_xs[T][nq:] == 0)  # zero terminal velocity

4. Define the dynamics, by both enforcing that acceleration matches ABA (var_a = ABA(q,v,tau)), and that the next state results from the integration of the acceleration.

In [16]:
# %load tp3/generated/free_integration
for t in range(T):
    opti.subject_to(caba(var_xs[t], var_us[t]) == var_as[t])
    opti.subject_to(cnext(var_xs[t], var_as[t]) == var_xs[t + 1])

5. Solve

In [17]:
# %load tp3/generated/free_ocp5
# SOLVE
opti.minimize(totalcost)
opti.solver("ipopt")  # set numerical backend
opti.callback(lambda i: displayScene(opti.debug.value(var_xs[-1][:nq])))

# Caution: in case the solver does not converge, we are picking the candidate values
# at the last iteration in opti.debug, and they are NO guarantee of what they mean.
try:
    sol = opti.solve_limited()
    sol_xs = [opti.value(var_x) for var_x in var_xs]
    sol_as = [opti.value(var_a) for var_a in var_as]
    sol_us = [opti.value(var_u) for var_u in var_us]
except:
    print("ERROR in convergence, plotting debug info.")
    sol_xs = [opti.debug.value(var_x) for var_x in var_xs]
    sol_as = [opti.debug.value(var_a) for var_a in var_as]
    sol_us = [opti.debug.value(var_u) for var_u in var_us]


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:     7818
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:     6015

Total number of variables............................:     1212
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      918
Total number of inequality c

And finally, display the result.

In [18]:
# %load tp3/generated/free_ocp6
print("***** Display the resulting trajectory ...")
displayScene(robot.q0, 1)
displayTraj([x[:nq] for x in sol_xs], DT)

***** Display the resulting trajectory ...


## Contact dynamics
Next, we now consider a humanoid robot in contact with the ground. We will only enforce here a bilateral contact, i.e. the solver will never "discover" that the robot can relax the contact and start walking. 

In [19]:
# %load tp3/generated/contact_talos
robot = robex.load("talos_legs")
# Open the viewer
viz = MeshcatVisualizer(robot)
viz.display(robot.q0)

# The pinocchio model is what we are really interested by.
model = robot.model
data = model.createData()

/home/matheecs/Code/jnrh2023/myenv/lib/python3.10/site-packages/cmeel.prefix/lib/python3.10/site-packages/example_robot_data/robots_loader.py:319: UserWarning: Deprecated member. Use Frame.parentJoint instead.
  if f.parent < legMaxId:


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


In [20]:
from types import SimpleNamespace

In [21]:
# %load tp3/generated/contact_frames
Mtarget = pin.SE3(pin.utils.rotate("y", 3),
                  np.array([-0.1, 0.2, 0.45094]))  # x,y,z
contacts = [SimpleNamespace(name="left_sole_link",
                            type=pin.ContactType.CONTACT_6D)]
endEffectorFrameName = "right_sole_link"

### Using the constraint dynamics solver
Our model now is in contact with the ground. A first solution is to extend the previous notebook: consider an explicit motion constraint on the acceleration, and as above, introduce the joint torques as a decision variable. Then we also have to decide the contact forces. This works, yet it implies many more variables; it is not easy to generalize, as new contacts would imply to change the structure of the trajectory optimization problem; and the efficiency of the trajectory solver to handle the dynamics is not ideal. 

Instead, we will change the dynamics to directly account for the contact constraint at that level. Then, the contact forces and the contact motion constraint are implicit for the trajectory solver, and the forces become a function of the torque rather than an explicit variable. 

For that, we will use the pin.constraintDynamics function of Pinocchio. This method is described in details in the paper of [Justin Carpentier (2021)](https://inria.hal.science/hal-03271811/file/rss-proximal-and-sparse.pdf). From a syntax point of view, it works nearly as ABA: you just need to specify the contact models.

Justin Carpentier et al (2021). Proximal and sparse resolution of constrained dynamic equations. In Robotics: Science and Systems.



In [22]:
# %load tp3/generated/contact_framesId
endEffector_ID = model.getFrameId(endEffectorFrameName)
for c in contacts:
    c.id = model.getFrameId(c.name)
    assert c.id < len(model.frames)
    c.jid = model.frames[c.id].parentJoint
    c.placement = model.frames[c.id].placement
    c.model = pin.RigidConstraintModel(c.type, model, c.jid, c.placement)
contact_models = [c.model for c in contacts]

Here, the pin.RigidConstraintModel is used to store the contact type (3D or 6D implemented for now), the joint ID, and the placement of the contact location on the joint. Additionally, you can specify the location of the contact in the world if you want to use a Baumgart position corrector (we will not).

In [23]:
# %load tp3/generated/contact_contact_solver
# Baumgart correction
Kv = np.array([20.] * 6)
Kp = np.array([0.] * 6)
# Tuning of the proximal solver (minimal version)
prox_settings = pin.ProximalSettings(0, 1e-6, 1)

In [24]:
# %load tp3/generated/contact_contact_setting
contact_datas = [c.createData() for c in contact_models]
for c in contact_models:
    c.corrector.Kd = Kv
    c.corrector.Kp = Kp

The constraint solver is a proximal solver (see the algorithm explained by Justin Carpentier 2021), which we will use minimally here as defined by the proximal settings (using a single iteration). You can call it like this:

In [25]:
pin.initConstraintDynamics(model, data, contact_models)
q = robot.q0.copy()
v = np.zeros(model.nv)
tau = np.zeros(model.nv)
pin.constraintDynamics(model, data, q, v, tau, contact_models, contact_datas)

array([ -1.09796479,  -1.37267382,  -7.88771551,   8.20606128,
        -3.76565864,   1.37070481,  -1.37070481,  -9.49136458,
       -19.27689169,  49.77986436, -26.73731403,   1.28530331,
        -2.43383966,  -9.04718988,   5.05175259,  -3.13190861,
         0.16928539,  -0.19395264])

### Formulation of the problem

The problem is then very similar to the previous one. We simply need to take care to the constraint dynamics with the appropriate algorithm. The contact forces are then a hiden quantity inside the dynamics. We also need to take care to the underactuation which states that their are no direct actuation of the basis. This can be enforced by setting the torques as:
$$\tau = [0_6, u]$$

The problem can then be written as follow.

Decide:
- $X = [ x_0 ... x_T ]$ the state trajectory (with $x = (robot.q0 + dq, v)$)
- $A = [a_0 ... a_{T-1} ]$ the accelerations
- $U = [u_0 ... u_{T-1} ]$ the controls (actuated joint torques)

Minimizing: $\sum_{t=0}^{T-1} v_t^2  + a_t^2 + e(q_T)^2$

Subject to:
- $x_0 = (robot.q0, 0)$
- $\forall t=0..T-1$
$$ \tau = [0_6,u_t]$$
$$ a_t = ABA(q_t,v_t,\tau_t) $$
$$ v_{t+1} = v_t + a_t \Delta t$$
$$ q_{t+1} = q_t \oplus v_{t+1} \Delta t$$

As previously, we need to define some helpers to get around the SX/MX syntax.

In [26]:
# %load tp3/generated/contact_helpers
# --- Casadi helpers
cmodel = cpin.Model(model)
cdata = cmodel.createData()
ccontact_models = [cpin.RigidConstraintModel(c) for c in contact_models]
ccontact_datas = [c.createData() for c in ccontact_models]
cprox_settings = cpin.ProximalSettings(
    prox_settings.absolute_accuracy, prox_settings.mu, prox_settings.max_iter
)
cpin.initConstraintDynamics(cmodel, cdata, ccontact_models)

nq = model.nq
nv = model.nv
nx = nq + nv
ndx = 2 * nv
cx = casadi.SX.sym("x", nx, 1)
cdx = casadi.SX.sym("dx", nv * 2, 1)
cq = cx[:nq]
cv = cx[nq:]
caq = casadi.SX.sym("a", nv, 1)
ctauq = casadi.SX.sym("tau", nv, 1)

# Compute kinematics casadi graphs
cpin.constraintDynamics(cmodel, cdata, cq, cv, ctauq,
                        ccontact_models, ccontact_datas)
cpin.forwardKinematics(cmodel, cdata, cq, cv, caq)
cpin.updateFramePlacements(cmodel, cdata)

Here the dynamics is evaluated using the constraint algorithm.
As the robot has a free floating basis, we will use the integration on the state space $\mathcal{X}$ to take care of the Lie group structure.

In [27]:
# %load tp3/generated/contact_integrate
# Sym graph for the integration operation x,dx -> x(+)dx = [model.integrate(q,dq),v+dv]
cintegrate = casadi.Function(
    "integrate",
    [cx, cdx],
    [casadi.vertcat(cpin.integrate(cmodel, cx[:nq],
                    cdx[:nv]), cx[nq:] + cdx[nv:])],
)

The Euler integration is defined as previously.

In [28]:
# %load tp3/generated/contact_cnext
# Sym graph for the integration operation x' = [ q+vDT+aDT**2, v+aDT ]
cnext = casadi.Function(
    "next",
    [cx, caq],
    [
        casadi.vertcat(
            cpin.integrate(cmodel, cx[:nq], cx[nq:] * DT + caq * DT**2),
            cx[nq:] + caq * DT,
        )
    ],
)

The acceleration resulting from the forward dynamics will be used to enforce the slack variable $A$.

In [29]:
# %load tp3/generated/contact_aba
# Sym graph for the aba operation
caba = casadi.Function("fdyn", [cx, ctauq], [cdata.ddq])

Finally, the operational target is defined by:

In [30]:
# %load tp3/generated/contact_error
# Sym graph for the operational error
error_tool = casadi.Function(
    "etool3", [cx], [
        cdata.oMf[endEffector_ID].translation - Mtarget.translation]
)

### Implementation of the OCP

You can now implement the problem. Follow the steps.

1. Define the decision variables $X,A,U$. Remember that the states $x$ are implemented as small steps $dx$ from a reference state $x_0 = [robot.q0,0_{nv} ]$.

In [31]:
# %load tp3/generated/contact_ocp1
opti = casadi.Opti()
var_dxs = [opti.variable(ndx) for t in range(T + 1)]
var_as = [opti.variable(nv) for t in range(T)]
var_us = [opti.variable(nv - 6) for t in range(T)]
var_xs = [
    cintegrate(np.concatenate([robot.q0, np.zeros(nv)]), var_dx) for var_dx in var_dxs
]

2. Compute the cost.

In [32]:
# %load tp3/generated/contact_ocp2
totalcost = 0
# Define the running cost
for t in range(T):
    totalcost += 1e-3 * DT * casadi.sumsqr(var_xs[t][nq:])
    totalcost += 1e-4 * DT * casadi.sumsqr(var_as[t])
totalcost += 1e1 * casadi.sumsqr(error_tool(var_xs[T]))

3. Enforce the boundary conditions (on $x_0$, and also on $v_T$ if you like).

In [33]:
# %load tp3/generated/contact_ocp3
opti.subject_to(var_xs[0][:nq] == robot.q0)
opti.subject_to(var_xs[0][nq:] == 0)  # zero initial velocity
opti.subject_to(var_xs[T][nq:] == 0)  # zero terminal velocity

4. Enforce the integration, (i) by setting $\tau$, then (ii) computing the acceleration by inverse dynamics and finally (iii) enforcing Euler integration.

In [34]:
# %load tp3/generated/contact_integration
for t in range(T):
    tau = casadi.vertcat(np.zeros(6), var_us[t])
    opti.subject_to(caba(var_xs[t], tau) == var_as[t])
    opti.subject_to(cnext(var_xs[t], var_as[t]) == var_xs[t + 1])

5. Solve the problem

In [35]:
# %load tp3/generated/contact_ocp5
# SOLVE
opti.minimize(totalcost)
opti.solver("ipopt")  # set numerical backend
opti.callback(lambda i: displayScene(opti.debug.value(var_xs[-1][:nq])))

# Caution: in case the solver does not converge, we are picking the candidate values
# at the last iteration in opti.debug, and they are NO guarantee of what they mean.
try:
    sol = opti.solve_limited()
    sol_xs = [opti.value(var_x) for var_x in var_xs]
    sol_as = [opti.value(var_a) for var_a in var_as]
    sol_us = [opti.value(var_u) for var_u in var_us]
except:
    print("ERROR in convergence, plotting debug info.")
    sol_xs = [opti.debug.value(var_x) for var_x in var_xs]
    sol_as = [opti.debug.value(var_a) for var_a in var_as]
    sol_us = [opti.debug.value(var_u) for var_u in var_us]

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:    55278
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:    46128

Total number of variables............................:     3336
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:     2805
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  2.9298679e+00 4.98e+01 9.02e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

You can finally display the result.

In [37]:
# %load tp3/generated/contact_ocp6
print("***** Display the resulting trajectory ...")
displayScene(robot.q0, 1)
displayTraj([x[:nq] for x in sol_xs], DT)

***** Display the resulting trajectory ...
